# wps_bccaq
A process that runs the [bccaq.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/BCCAQ.R) function from ClimDown.

Bias Correction/Constructed Analogues with Quantile mapping reordering (BCCAQ) is composed of the following steps.
- Constructed Analogues (CA)
- Climate Imprint (CI)
- Quantile Delta Mapping (QDM)
- Rerank


In [1]:
from birdy import WPSClient
from netCDF4 import Dataset
from wps_tools.testing import get_target_url
from wps_tools.utils import copy_http_content
from tempfile import NamedTemporaryFile
import requests
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [3]:
chickadee = WPSClient(url)

ServiceException: <?xml version="1.0" encoding="utf-8"?>
<ExceptionReport version="1.0.0"
    xmlns="http://www.opengis.net/ows/1.1"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.opengis.net/ows/1.1 http://schemas.opengis.net/ows/1.1.0/owsExceptionReport.xsd">
    <Exception exceptionCode="NoApplicableCode" locator="NotAcceptable">
        <ExceptionText>Request failed: HTTPConnectionPool(host=&#x27;docker-dev03.pcic.uvic.ca&#x27;, port=8102): Max retries exceeded with url: /wps/wps?service=WPS&amp;request=GetCapabilities&amp;version=1.0.0 (Caused by NewConnectionError(&#x27;&lt;urllib3.connection.HTTPConnection object at 0x7f77927c5c50&gt;: Failed to establish a new connection: [Errno 111] Connection refused&#x27;))</ExceptionText>
    </Exception>
</ExceptionReport>

### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [ ]:
# NBVAL_IGNORE_OUTPUT
chickadee.bccaq?

### Run the bccaq process

In [ ]:
# NBVAL_IGNORE_OUTPUT
gcm_file = "tests/data/tiny_gcm.nc"
obs_file = "tests/data/tiny_obs.nc"
var = "tasmax"
end_date = "1972-12-31"
out_file = "out.nc"

output = chickadee.bccaq(
    gcm_file=gcm_file, 
    obs_file=obs_file, 
    var=var, 
    end_date=end_date,
    out_file=out_file
)
    
output.get()[0]

### Test for expected output

In [ ]:
with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    output_data = Dataset(copy_http_content(output.get()[0], tmp_file))
    expected_data = Dataset('tests/data/bccaq_expected_output.nc')
    for key, value in expected_data.dimensions.items():
        assert str(output_data.dimensions[key]) == str(value)